#### Link of the Pycaret docs : https://pycaret.readthedocs.io/en/latest/api/time_series.html

In [1]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import imageio
import os
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
import pmdarima as pm


In [2]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbmilho_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date  producao  produtividade     area
0  1977   19256.0           1632  11797.3
1  1978   14017.0           1276  10985.1
2  1979   16514.0           1461  11304.8
3  1980   19435.0           1665  11669.9
4  1981   21284.0           1752  12147.1
5  1982   21605.0           1692  12771.1
6  1983   19015.0           1631  11658.2
7  1984   21178.0           1735  12205.6
8  1985   21175.0           1773  11940.2
9  1986   20265.0           1549  13083.5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           47 non-null     int64  
 1   producao       47 non-null     float64
 2   produtividade  47 non-null     int64  
 3   area           47 non-null     float64
dtypes: float64(2), int64(2)
memory usage: 1.6 KB


### Avaliando a produção de soja

In [4]:
df.drop(['produtividade','area'], axis = 1,inplace=True)
df.head(10)


,date,producao
0,1977,19256.0
1,1978,14017.0
2,1979,16514.0
3,1980,19435.0
4,1981,21284.0
5,1982,21605.0
6,1983,19015.0
7,1984,21178.0
8,1985,21175.0
9,1986,20265.0


In [5]:
df.set_index('date',inplace=True)

In [6]:
df=df.sort_index(axis=0)
print(df.head(10))

      producao
date          
1977   19256.0
1978   14017.0
1979   16514.0
1980   19435.0
1981   21284.0
1982   21605.0
1983   19015.0
1984   21178.0
1985   21175.0
1986   20265.0


fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. When integer is passed it means N continuous points in the future without any gap. If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [7]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,42
1,Target,producao
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(47, 1)"
5,Transformed data shape,"(47, 1)"
6,Transformed train set shape,"(46, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [10]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Checking Train-Test Split

In [10]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Comparing Models

The most basic assumption of time series forecasting is that past patterns continue into the future. 
Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, 
the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

In [8]:
top4 = exp.compare_models(n_select = 4)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:13:50
Status,. . . . . . . . . . . . . . . . . .,Loading Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
ets,ETS,1.6405,1.1662,10244.3726,10244.3726,0.1072,0.1022,0.0333
exp_smooth,Exponential Smoothing,1.6454,1.1698,10278.5309,10278.5309,0.1071,0.1027,0.1133
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,2.0805,1.4832,13134.4397,13134.4397,0.1235,0.1338,0.1000
theta,Theta Forecaster,2.1441,1.5267,13468.5276,13468.5276,0.1340,0.1355,0.0300
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,2.2507,1.6049,14221.0769,14221.0769,0.1324,0.1463,0.1733
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,2.2560,1.6087,14250.5524,14250.5524,0.1335,0.1461,0.0767
auto_arima,Auto ARIMA,2.3010,1.6368,14407.9862,14407.9862,0.1461,0.1452,0.1333
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,2.3309,1.6628,14745.3764,14745.3764,0.1364,0.1516,0.0767
naive,Naive Forecaster,2.3324,1.6622,14688.9667,14688.9667,0.1443,0.1495,2.1233
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,2.3719,1.6909,14959.5009,14959.5009,0.1441,0.1529,0.0433


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data. 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

## Difference Plot Using Order

In [12]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":10})

### theta 

In [ ]:
theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
#exp.plot_model(estimator = theta, 
#               plot = 'forecast', 
#               data_kwargs = {'fh' : 9},
#               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(theta, fh = 9)

###  ADA

In [13]:
ada_cds_dt = exp.create_model('ada_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ada_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## exp_smooth

In [14]:
exp_smooth  = exp.create_model('exp_smooth')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,2.1306,2.1306,6775970.0000,6775970.0000,0.0593,0.0576
1,2019.0000,1.7762,1.7729,6017763.4792,6017763.4792,0.0494,0.0506
2,2020.0000,2.7832,2.5759,13248251.7917,13248251.7917,0.0983,0.1034
Mean,nan,2.2300,2.1598,8680661.7569,8680661.7569,0.0690,0.0705
SD,nan,0.4171,0.3285,3244572.7326,3244572.7326,0.0211,0.0234


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## ets

In [15]:

ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## ALL GRAPHS

In [17]:
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ada_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})              

## Avaliando a Produtividade

In [18]:
#função para tratar campo data                                                                                                                                                                      
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbmilho_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date   producao  disponibilidade   tipo
0  2017  114732101         17427206   soja
1  2018  117912450         16126368   soja
2  2019  114316829         17515920   soja
3  2020  121820949         18258981   soja
4  2021  134799179         18984300   soja
5  2022  120701031         21037669   soja
6  2017   97910658         33959879  milho
7  2018   82366531         34777936  milho
8  2019  101126409         35895207  milho
9  2020  103990935         37191638  milho


In [19]:
df.drop(['producao','area'], axis = 1,inplace=True)
df.head(10)

,date,producao,tipo
6,2017,97910658,milho
7,2018,82366531,milho
8,2019,101126409,milho
9,2020,103990935,milho
10,2021,88272116,milho
11,2022,109420717,milho


In [21]:
df.set_index('date',inplace=True)

In [22]:
df=df.sort_index(axis=0)
print(df.head(10))

       producao
date           
2017   97910658
2018   82366531
2019  101126409
2020  103990935
2021   88272116
2022  109420717


In [23]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,15
1,Target,producao
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(6, 1)"
5,Transformed data shape,"(6, 1)"
6,Transformed train set shape,"(5, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [24]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})


In [25]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})


In [26]:
top4 = exp.compare_models(n_select = 4)


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data.
 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

In [27]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":10})


####  ada

In [28]:
ada_cds_dt = exp.create_model('ada_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ada_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(ada_cds_dt, fh = 9)

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2022,101830605.0
2023,104695131.0
2024,88976312.0
2025,102534801.0
2026,105399327.0
2027,89680508.0
2028,103238997.0
2029,106103523.0
2030,90384704.0


In [29]:
xgboost_cds_dt = exp.create_model('xgboost_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = xgboost_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(ada_cds_dt, fh = 9)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,2.2069,2.2069,34304005.0000,34304005.0000,0.3392,0.4085
1,2019.0000,1.0733,1.0686,18408649.6667,18408649.6667,0.1770,0.1942
2,2020.0000,1.5674,1.3711,19418889.7500,19418889.7500,0.2200,0.1982
Mean,nan,1.6158,1.5489,24043848.1389,24043848.1389,0.2454,0.2670
SD,nan,0.4641,0.4814,7266739.7718,7266739.7718,0.0686,0.1001


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2022,101830605.0
2023,104695131.0
2024,88976312.0
2025,102534801.0
2026,105399327.0
2027,89680508.0
2028,103238997.0
2029,106103523.0
2030,90384704.0


#### gbr_cds_dt   Gradient Boosting w/ Cond. Deseasonalize & Detrending

In [30]:
gbr_cds_dt = exp.create_model('gbr_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = gbr_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,2.2069,2.2069,34304005.0000,34304005.0000,0.3392,0.4085
1,2019.0000,1.0733,1.0686,18408425.2094,18408425.2094,0.1770,0.1942
2,2020.0000,1.5674,1.3711,19418771.3590,19418771.3590,0.2200,0.1982
Mean,nan,1.6158,1.5489,24043733.8561,24043733.8561,0.2454,0.2670
SD,nan,0.4641,0.4814,7266822.9094,7266822.9094,0.0686,0.1001


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


#### rf_cds_dt   Random Forest w/ Cond. Deseasonalize & Detrending

In [31]:
rf_cds_dt = exp.create_model('rf_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = rf_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.



## ada_cds_dt   AdaBoost w/ Cond. Deseasonalize & Detrendin


In [32]:
ada_cds_dt = exp.create_model('ada_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ada_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


In [33]:
ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 9},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(ets, fh = 9)

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2022,8.633219e+07
2023,8.229880e+07
2024,7.826540e+07
2025,7.423200e+07
2026,7.019861e+07
2027,6.616521e+07
2028,6.213182e+07
2029,5.809842e+07
2030,5.406503e+07


### ALL GRAPHS

In [34]:
exp.plot_model(estimator = xgboost, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = gbr_cds_dt , plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = rf_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ada_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

NameError: name 'xgboost' is not defined

In [ ]:
## Avaliando a Area

In [ ]:
#função para tratar campo data                                                                                                                                                                      
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbmilho_br.xls', sheet_name='Planilha1')
print(df.head(10))



In [ ]:
df.drop(['producao','produtividade'], axis = 1,inplace=True)
df.set_index('date',inplace=True)
df=df.sort_index(axis=0)
print(df.head(10))

In [ ]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

In [ ]:
#exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
#exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
top4 = exp.compare_models(n_select = 4)


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data.
 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.


In [ ]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":10})